#一、构建神经网络
- 神经网络由对数据执行操作的层/模块组成。
- torch.nn 命名空间提供了构建您自己的神经网络所需的所有构建块。 
- PyTorch 中的每个模块都是 nn.Module 的子类。神经网络本身就是一个模块，由其他模块（层）组成。
- 这种嵌套结构允许轻松构建和管理复杂的架构。

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1、获取执行资源
- 我们希望能够在 GPU 或 MPS 等硬件加速器（如果可用）上训练我们的模型。
- 让我们检查一下 torch.cuda 或 torch.backends.mps 是否可用，否则我们使用 CPU。

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


## 2、自定义类
- 我们通过子类化 nn.Module 来定义神经网络，并在 __init__ 中初始化神经网络层。
- 每个nn.Module子类都实现了forward方法中对输入数据的操作。

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        """
        在PyTorch中，nn.Flatten()是一个模块，用于将一个多维张量（tensor）转换为一维或更少维度的张量，
        通常是将一个多维的输出转换为二维张量，以便可以将其传递给全连接层（fully connected layers）或其他线性层。
        当你在神经网络模型中定义self.flatten = nn.Flatten()时，你实际上是在模型中加入了一个平坦化层。
        这个层的作用是将输入的任何形状（除了第一个维度，通常是batch size）展平成一个连续的向量。
        例如，假设你有一个卷积神经网络（CNN），其最后一层的输出是形状为 (batch_size, channels, height, width) 的四维张量。
        如果你想将这个输出传递给一个全连接层，你需要先将它转换成二维张量，形状通常是 (batch_size, features)，
        其中 features 是所有通道、高度和宽度的乘积。
        """
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


## 3、模型层
- 让我们分解 FashionMNIST 模型中的各个层。
- 为了说明这一点，我们将采用 3 张大小为 28x28 的图像的小批量样本，看看当我们将其传递到网络时会发生什么。

In [6]:
input_imageinput_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# nn.Flatten
# 我们初始化 nn.Flatten 层，将每个 2D 28x28 图像转换为 784 个像素值的连续数组（维持小批量维度（在 dim=0 时））。
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
# nn.Linear
# 线性层是一个使用其存储的权重和偏差对输入应用线性变换的模块。
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
# 变换以后 从[3, 784]变成[3, 20]
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
# nn.ReLU
# 非线性激活在模型的输入和输出之间创建复杂的映射。它们在线性变换后应用以引入非线性，帮助神经网络学习各种现象。
# 在此模型中，我们在线性层之间使用 nn.ReLU，但还有其他激活可以在模型中引入非线性。
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0319,  0.0195, -0.3762, -0.1158, -0.5517, -0.1391, -0.4461,  0.1989,
          0.0841,  0.3436, -0.2756, -0.1493, -0.5748,  0.1428,  0.5251,  0.2716,
          0.1466,  0.1626, -0.1141, -0.2382],
        [-0.1596, -0.0451, -0.4958, -0.2541, -0.0168, -0.2249, -0.3163,  0.1529,
          0.3354,  0.3275,  0.0979,  0.2216, -0.4336,  0.2924, -0.1696,  0.3141,
          0.1777,  0.2210, -0.4960, -0.0245],
        [-0.3376,  0.1092, -0.3129,  0.1933,  0.0222, -0.2658, -0.5016,  0.1466,
          0.3429,  0.4398,  0.1470, -0.1319, -0.3944,  0.3044,  0.0208,  0.1931,
          0.1080,  0.0395, -0.1340, -0.3612]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0319, 0.0195, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1989, 0.0841,
         0.3436, 0.0000, 0.0000, 0.0000, 0.1428, 0.5251, 0.2716, 0.1466, 0.1626,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1529, 0.3354,
         0.3275, 0.0979, 0.2216, 0.0000, 0.2924, 0.00

In [11]:
# nn.Sequential
# nn.Sequential 是模块的有序容器。数据按照定义的相同顺序传递通过所有模块。您可以使用顺序容器来组合一个快速网络，例如 seq_modules。
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
logits

tensor([[-0.0400,  0.5057,  0.1896, -0.2307,  0.1639, -0.1911, -0.2428, -0.2212,
          0.0345,  0.2166],
        [ 0.0383,  0.2880,  0.1028, -0.1620, -0.0335, -0.1145, -0.2037, -0.1511,
         -0.1040,  0.1769],
        [ 0.1206,  0.3520,  0.0698, -0.0723,  0.1250, -0.0723, -0.1880, -0.1598,
         -0.0738,  0.2145]], grad_fn=<AddmmBackward0>)

In [15]:
# nn.Softmax
# 神经网络的最后一个线性层返回 logits - [-infty, infty] 中的原始值 - 被传递到 nn.Softmax 模块。
# Logits 缩放为值 [0, 1]，表示模型对每个类别的预测概率。 dim 参数指示值总和必须为 1 的维度。
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0916, 0.1581, 0.1152, 0.0757, 0.1123, 0.0788, 0.0748, 0.0764, 0.0987,
         0.1184],
        [0.1043, 0.1339, 0.1113, 0.0854, 0.0971, 0.0895, 0.0819, 0.0863, 0.0905,
         0.1198],
        [0.1078, 0.1359, 0.1025, 0.0889, 0.1083, 0.0889, 0.0792, 0.0814, 0.0888,
         0.1184]], grad_fn=<SoftmaxBackward0>)

## 4、模型参数
- 神经网络内的许多层都是参数化的，即具有在训练期间优化的相关权重和偏差。
- 子类化 nn.Module 会自动跟踪模型对象中定义的所有字段，并使用模型的parameters()或named_parameters()方法访问所有参数。

In [16]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0024, -0.0120,  0.0333,  ...,  0.0245,  0.0288,  0.0239],
        [-0.0347, -0.0269,  0.0138,  ..., -0.0038, -0.0128, -0.0139]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0157, -0.0299], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0442,  0.0249,  0.0031,  ...,  0.0391, -0.0301,  0.0077],
        [-0.0416,  0.0282, -0.0170,  ..., -0.0097,  0.0086, -0.0353]],
       device='cuda:0', grad_fn=<Sl